In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image
import os
import copy
from torch.utils.data import DataLoader
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics import balanced_accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import datasets
from sklearn.model_selection import train_test_split
import cv2
from typing import List

LABELS_Severity = {35: 0,
                   43: 0,
                   47: 1,
                   53: 1,
                   61: 2,
                   65: 2,
                   71: 2,
                   85: 2}

def normalize_np(image, mean, std):
    grayscale_image = np.array(image.convert('L'))
    return (grayscale_image - mean) / std

mean = 0.1706
std = 0.2112
target_size = (224, 224) #(112, 112)
train_transform = transforms.Compose([
    transforms.Resize(size=target_size),
    transforms.Lambda(lambda x: normalize_np(x, mean, std)),
])

test_transform = transforms.Compose([
    transforms.Resize(size=target_size),
    transforms.Lambda(lambda x: normalize_np(x, mean, std)),
])

In [ ]:
class LaplacianPyramidFusion:
    def __init__(self, image_paths, levels=5, device='cuda:0'):
        self.image_paths = image_paths
        self.levels = levels
        self.device = device

    def read_images(self):
        images = []
        for path in self.image_paths:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = torch.from_numpy(img).unsqueeze(0).unsqueeze(0).to(self.device)
            images.append(img)        
        return images      

    def gaussian_pyramid(self, image):
        image = image.float()  # Convert image to float32 format
        g_pyr = [image]
        for _ in range(self.levels-1):
            image = torch.nn.functional.avg_pool2d(image, kernel_size=2, stride=2, padding=0)
            # image = cv2.pyrDown(image)
            g_pyr.append(image)
        return g_pyr

    def laplacian_pyramid(self, g_pyr):
        l_pyr = [g_pyr[-1]]
        for i in range(len(g_pyr)-1, 0, -1):
            size = (g_pyr[i - 1].shape[2], g_pyr[i - 1].shape[1])
            expanded = torch.nn.functional.interpolate(g_pyr[i], size=size, mode='bilinear', align_corners=False)
            laplacian = g_pyr[i - 1] - expanded
            l_pyr.append(laplacian)

            # size = (g_pyr[i-1].shape[1], g_pyr[i-1].shape[0])
            # expanded = cv2.pyrUp(g_pyr[i], dstsize=size)
            # laplacian = cv2.subtract(g_pyr[i-1], expanded)
            # l_pyr.append(laplacian)
        return l_pyr[::-1]

    def blend_pyramids(self, pyramids):
        blended_pyr = []
        for images in zip(*pyramids):
            blended_img = torch.zeros_like(images[0], dtype=torch.float32)
            # blended_img = np.zeros_like(images[0], dtype=np.float32)
            for img in images:
                blended_img += img
                # blended_img += img.astype(np.float32)
            blended_img /= len(images)
            blended_pyr.append(blended_img)
        return blended_pyr

    def reconstruct_image(self, l_pyr):
        image = l_pyr[-1]
        for i in range(len(l_pyr) - 2, -1, -1):
            size = (l_pyr[i].shape[2], l_pyr[i].shape[1])
            expanded = torch.nn.functional.interpolate(image, size=size, mode='bilinear', align_corners=False)
            image = expanded + l_pyr[i]
        return image.squeeze().cpu().numpy().astype(np.uint8)

        #     size = (l_pyr[i].shape[1], l_pyr[i].shape[0])
        #     expanded = cv2.pyrUp(image, dstsize=size)
        #     image = cv2.add(expanded, l_pyr[i])
        # return image.astype(np.uint8)

    def fuse(self):
        images = self.read_images()
        g_pyrs = [self.gaussian_pyramid(img) for img in images]
        l_pyrs = [self.laplacian_pyramid(g_pyr) for g_pyr in g_pyrs]
        blended_pyr = self.blend_pyramids(l_pyrs)
        fused_image = self.reconstruct_image(blended_pyr)
        return fused_image

In [ ]:
def blend_images(img1, img2, alpha):
    # # Convert the images to the same size
    # img1_resized = cv2.resize(img1, (img2.shape[1], img2.shape[0]))

    # # Perform alpha blending
    # blended = cv2.addWeighted(img1_resized, alpha, img2, 1 - alpha, 0)

    #Try-2
    # Convert the images to the same size
    img2_resized = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    # Ensure both images have the same number of dimensions
    if len(img1.shape) != len(img2_resized.shape):
        if len(img1.shape) == 3 and len(img2_resized.shape) == 2:
            img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        elif len(img1.shape) == 2 and len(img2_resized.shape) == 3:
            img2_resized = cv2.cvtColor(img2_resized, cv2.COLOR_BGR2GRAY)

    # Perform alpha blending
    blended = cv2.addWeighted(img1, alpha, img2_resized, 1 - alpha, 0)

    return blended

# Add this function to combine edge-detected images
def combine_edges(edges: List[np.ndarray]) -> np.ndarray:
    combined = np.zeros_like(edges[0], dtype=np.uint8)
    for edge in edges:
        combined = cv2.addWeighted(combined, 1, edge, 1, 0)
    return combined

# Function to read image, apply Canny edge detection, and return the result
def apply_canny_edge(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 200, 500)
    return edges

# Function to create a grid image from a list of edge images
# def create_grid_image(edge_images, grid_shape=(7, 7)):
#     rows = []
#     for r in range(grid_shape[0]):
#         row = np.hstack(edge_images[r*grid_shape[1]:(r+1)*grid_shape[1]])
#         rows.append(row)
#     grid_image = np.vstack(rows)
#     return grid_image
def create_grid_image(edge_images, grid_shape=(7, 7), new_size=(64, 64)):
    rows = []
    for r in range(grid_shape[0]):
        resized_images = []
        for img in edge_images[r*grid_shape[1]:(r+1)*grid_shape[1]]:
            resized_img = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
            resized_images.append(resized_img)
        row = np.hstack(resized_images)
        rows.append(row)
    grid_image = np.vstack(rows)
    return grid_image

In [ ]:
class OCTDataset(Dataset):
    def __init__(self, annot=None, subset='train', transform=None, device='cpu'):
        if subset == 'train':
            self.annot = pd.read_csv("/content/drive/MyDrive/FML_Project/df_prime_train.csv")
        elif subset == 'test':
            self.annot = pd.read_csv("/content/drive/MyDrive/FML_Project/df_prime_test.csv")

        # Extract "Patient_ID" and "Week_Num" columns
        # print("Before Pairing ", len(self.annot))
        self.patient_ids = self.annot["Patient_ID"]
        self.week_nums = self.annot["Week_Num"]
        self.patient_ids = self.annot["Patient_ID"]
        self.annot['Severity_Label'] = [LABELS_Severity[drss] for drss in copy.deepcopy(self.annot['DRSS'].values)]
        self.drss_class = self.annot['Severity_Label']


        # Create unique pairs of values
        self.unique_pairs = set(zip(self.patient_ids, self.week_nums, self.drss_class))

        # Print the unique pairs
        # print(len(self.unique_pairs))
        # for pair in self.unique_pairs:
        #     print(pair)

        self.root = os.path.expanduser("/content/drive/MyDrive/FML_Project/")
        self.transform = transform
        self.nb_classes=len(np.unique(list(LABELS_Severity.values())))
        self.path_list = self.annot['File_Path'].values

        self._labels = [pair[2] for pair in self.unique_pairs]
        # self._labels = self.annot['Severity_Label'].values
        assert len(self.unique_pairs) == len(self._labels)
        
        max_samples = int(len(self._labels)) #32 #int(len(self._labels)/2)
        self.max_samples = max_samples
        self.device = device

    def __getitem__(self, index):
        # Get the Patient_ID and Week_Num from the indexed element in unique_pairs
        patient_id, week_num, target = list(self.unique_pairs)[index]

        # Filter the annot DataFrame to select rows that match the Patient_ID and Week_Num
        filtered_df = self.annot[(self.annot['Patient_ID'] == patient_id) & (self.annot['Week_Num'] == week_num)]

        # Extract the file paths from the filtered DataFrame and return them as a list
        # file_paths = filtered_df['File_Path'].values.tolist()
        file_paths = [self.root + file_path for file_path in filtered_df['File_Path'].values.tolist()]
        output_dir = os.path.dirname(file_paths[0])

        # LaplacianPyramidFusion
        #fusion = LaplacianPyramidFusion(file_paths)
        #fused_image = fusion.fuse()
        #cv2.imwrite(os.path.join(output_dir, f"fused_image_level_5.jpg"), fused_image)
        #print("fused op:"+os.path.join(output_dir, f"fused_image_level_5.jpg"))

        # Alpha blending
        # Blend the images sequentially
        # blended = cv2.imread(file_paths[0], cv2.IMREAD_UNCHANGED)
        # for i in range(1, len(file_paths)):
        #     alpha = 1 - (i / len(file_paths)) # more weightage to first image encountered
        #     alpha = 1 / len(file_paths) #equal weightage for each image
        #     next_image = cv2.imread(file_paths[i], cv2.IMREAD_UNCHANGED)
        #     blended = blend_images(blended, next_image, alpha)
        # output_filename = f'ab_final.png'
        # output_filepath = os.path.join(output_dir, output_filename)
        # print("alpha op:"+output_filepath)
        # cv2.imwrite(output_filepath, blended)

        # Canny edges
        # Apply the Canny edge detection on each image in file_paths and store the results in a list
        # Check this path for other combinations:
        # /content/drive/MyDrive/FML_Project//Prime_FULL/01-027/W52/OS/cn_final_200_700.jpg
        # Optimal based on visual inspection: 200_500
        edge_images = [cv2.Canny(cv2.imread(file_path, cv2.IMREAD_GRAYSCALE), 300, 700) for file_path in file_paths]
        combined_edges = combine_edges(edge_images)
        output_filename = f'cn_trial_300_700.jpg'
        output_filepath = os.path.join(output_dir, output_filename)
        print("canny op:"+output_filepath)
        cv2.imwrite(output_filepath, combined_edges)

        # Sobel operator
        # Apply the Sobel-Scharr on each image in file_paths and store the results in a list
        # Check this path for other combinations:
        # /content/drive/MyDrive/FML_Project//Prime_FULL/01-027/W52/OS/cn_final_200_700.jpg
        # Optimal based on visual inspection: 200_500
        # edge_images = []
        # for file_path in file_paths:
        #     image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        #     #10,01-
        #     scharr_x = cv2.Scharr(image, cv2.CV_64F, 0, 1)
        #     scharr_y = cv2.Scharr(image, cv2.CV_64F, 0, 1)

        #     # Combine x and y gradients
        #     scharr = cv2.addWeighted(scharr_x, 0.5, scharr_y, 0.5, 0)
        #     # Convert to uint8 to have a similar range as Canny
        #     scharr = cv2.convertScaleAbs(scharr)
        #     edge_images.append(scharr)
        # combined_edges = combine_edges(edge_images)
        # output_filename = f'sobel_trial.jpg'
        # output_filepath = os.path.join(output_dir, output_filename)
        # print("Sobel op:"+output_filepath)
        # cv2.imwrite(output_filepath, combined_edges)

        # Canny edge on fusion image level 5
        # fused_image_l5_path = os.path.join(output_dir, f"fused_image_level_5.jpg")
        # fused_edge_image_l5 = cv2.Canny(cv2.imread(fused_image_l5_path, cv2.IMREAD_GRAYSCALE), 200, 500)
        # output_filename = f'fused_edge_image_l5.jpg'
        # output_filepath = os.path.join(output_dir, output_filename)
        # print("Canny edge on fusion image level 5 op:"+output_filepath)
        # cv2.imwrite(output_filepath, fused_edge_image_l5)        

        # Canny edge on alpha blended image
        # ab_image_path = os.path.join(output_dir, f"ab_final.png")
        # ab_edge_image = cv2.Canny(cv2.imread(ab_image_path, cv2.IMREAD_GRAYSCALE), 200, 500)
        # output_filename = f'ab_edge_image.jpg'
        # output_filepath = os.path.join(output_dir, output_filename)
        # print("Canny edge on alpha blended image op:"+output_filepath)
        # cv2.imwrite(output_filepath, ab_edge_image)   

        # Create 7x7 grid - plain - image
        # Sort image paths by file name in ascending order
        # sorted_image_paths = sorted(file_paths, key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
        # #print(sorted_image_paths)
        # # print(os.path.join(output_dir, f"grid_image.jpg"))
        # # Apply Canny edge detection on each sorted image
        # sorted_images = [cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) for image_path in sorted_image_paths]

        # # Calculate the number of placeholder images needed
        # num_placeholders = 49 - len(sorted_images)

        # # Create black background placeholder images and add them to sorted_images
        # if num_placeholders > 0:
        #     # Determine the size of the placeholder image based on the first image in the sorted_images list
        #     placeholder_size = sorted_images[0].shape if sorted_images else (64, 64)  # Assuming 64x64 if no images are found
            
        #     for _ in range(num_placeholders):
        #         placeholder_image = np.zeros(placeholder_size, dtype=np.uint8)
        #         sorted_images.append(placeholder_image)

        # # Create a 7x7 grid image
        # grid_image = create_grid_image(sorted_images, grid_shape=(7, 7))
        # # Save the grid image
        # cv2.imwrite(os.path.join(output_dir, f"grid_image.jpg"), grid_image)


        # # Create 7x7 grid - canny - image
        # # Sort image paths by file name in ascending order
        # sorted_image_paths = sorted(file_paths, key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
        # #print(sorted_image_paths)
        # # print(os.path.join(output_dir, f"grid_image_canny.jpg"))
        # # Apply Canny edge detection on each sorted image
        # sorted_edge_images = [apply_canny_edge(image_path) for image_path in sorted_image_paths]

        # # Calculate the number of placeholder images needed
        # num_placeholders = 49 - len(sorted_edge_images)

        # # Create black background placeholder images and add them to sorted_edge_images
        # if num_placeholders > 0:
        #     # Determine the size of the placeholder image based on the first image in the sorted_edge_images list
        #     placeholder_size = sorted_edge_images[0].shape if sorted_edge_images else (64, 64)  # Assuming 64x64 if no images are found
            
        #     for _ in range(num_placeholders):
        #         placeholder_image = np.zeros(placeholder_size, dtype=np.uint8)
        #         sorted_edge_images.append(placeholder_image)


        # # Create a 7x7 grid image
        # grid_image = create_grid_image(sorted_edge_images, grid_shape=(7, 7))
        # # Save the grid image
        # cv2.imwrite(os.path.join(output_dir, f"grid_image_canny.jpg"), grid_image)

        # print("File paths of ", patient_id, " , ", week_num, str(file_paths))
        # print(type(img), type(img_gray))

        if self.transform is not None:
            img_gray = self.transform(Image.fromarray(combined_edges))

        return img_gray, target

    def __len__(self):
        if self.max_samples is not None:
            return min(len(self._labels), self.max_samples)
        else:
            return len(self._labels)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Found device', device)
batch_size = 32

trainset = OCTDataset(subset='train', transform=train_transform, device=device)
testset = OCTDataset(subset='test', transform=test_transform, device=device)

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)
print(len(trainset), len(testset))

# for i in range(len(trainset)):
#   print(trainset[i][0].shape)
#   print("Train fusion done:", i)

# for i in range(len(testset)):
#   print(testset[i][0].shape)
#   print("Test fusion done:", i)

print(trainset[1][0].shape)


Found device cuda:0
495 163
canny op:/content/drive/MyDrive/FML_Project//Prime_FULL/01-027/W52/OS/cn_trial_300_700.jpg
(224, 224)
